In [1]:
import pandas as pd
import numpy as np 
import os
import altair as alt
import tabula 
import html5lib

In [20]:
pd.options.mode.chained_assignment = None

In [268]:
dfs = []
raw = []
files = [f for f in os.listdir("../data") if f.endswith('html')]

for file in files:
    target = pd.read_html(f"../data/{file}")[-1]
    raw.append(target)
    
    disabled = target.iloc[1:8, :]
    disabled['disability_status'] = 'disabled'
    vals = disabled[disabled['Disability Status/ Insurance Type'] == 'Uninsured'].values[0]
    pct_insured = 100 - float(vals[1])
    disabled_insured_row = ['Insured', pct_insured]
    disabled_insured_row.extend(vals[2:])
    disabled_insured_row = pd.DataFrame([disabled_insured_row], columns = disabled.columns)
    disabled = pd.concat([disabled, disabled_insured_row])
    
    not_disabled = target.iloc[9:, :]
    not_disabled['disability_status'] = 'not_disabled'
    vals = not_disabled[not_disabled['Disability Status/ Insurance Type'] == 'Uninsured'].values[0]
    pct_insured = 100 - float(vals[1])
    not_disabled_insured_row = ['Insured', pct_insured]
    not_disabled_insured_row.extend(vals[2:])
    not_disabled_insured_row = pd.DataFrame([not_disabled_insured_row], columns = not_disabled.columns)
    not_disabled = pd.concat([not_disabled, not_disabled_insured_row])
    
    
    target = pd.concat([not_disabled, disabled])
    target['year'] = int(file.split("_")[-1][:4])
    dfs.append(target)

    
df = pd.concat(dfs).reset_index(drop = True)
raw_df = pd.concat(raw).reset_index(drop = True)

In [377]:
df['percent_new'] = df['Percent'].astype(float) / 100.0
df = df.rename(columns = {'Disability Status/ Insurance Type': 'type'})

In [378]:
insured = df[(df.type == 'Insured') & (df.disability_status == 'disabled')]
uninsured = df[(df.type == 'Uninsured') & (df.disability_status == 'disabled')]
employer = df[(df.type == 'Employer/Union') & (df.disability_status == 'disabled')]
purchased = df[(df.type == 'Purchased') & (df.disability_status == 'disabled')]
medicare = df[(df.type == 'Medicare') & (df.disability_status == 'disabled')]
medicaid = df[(df.type == 'Medicaid') & (df.disability_status == 'disabled')]

In [379]:
disabled = df[(df.disability_status == 'disabled') & (df.type != 'Insured') & (df.type != 'Uninsured')]

all_uninsured = df[df.type == 'Uninsured']

In [446]:
#visualization 1
highlight = alt.selection_single(on = 'mouseover', fields = ['type'], nearest = True)
color_condition = alt.condition(highlight, alt.Color('type:N'), alt.value('gray'))

nearest_val = alt.selection_single(on = 'mouseover', nearest = True, fields = ['year'], empty = 'none')
opacity_condition = alt.condition(nearest_val, alt.value(1.0), alt.value(0.0))
text_condition = alt.condition(nearest_val, alt.Text('percent_new:Q'), alt.value(0.0))

aca_df = pd.DataFrame({'year': [2010]})
aca_effect_df = pd.DataFrame({'year': [2014]})

uninsured_rates = alt.Chart(all_uninsured).mark_line().encode(
    x = alt.X('year:O', axis = alt.Axis(title = 'Year')), 
    y = alt.Y('percent_new:Q', axis = alt.Axis(format = '%', title = 'Percent Uninsured')), 
    color = alt.Color('disability_status:O')
)

aca_passage = alt.Chart(aca_df).mark_rule(
    color = 'tomato', 
    size = 2.5
).encode(
    x = alt.X('year:O')
)

aca_passage_text = alt.Chart(aca_df).mark_text(
    lineBreak = r'\n',
    text = r"2010: Passage of \nthe Affordable Care Act",
    align = 'left',
    fontSize = 10, 
    dy = 135, 
    dx = -95
)

aca_effect = alt.Chart(aca_effect_df).mark_rule(
    color = 'tomato', 
    size = 2.5
).encode(
    x = alt.X('year:O')
)

aca_effect_text = alt.Chart(aca_effect_df).mark_text(
    lineBreak = r'\n',
    text = r'2014: Most ACA \nProvisions go into Effect', 
    align = 'left',
    fontSize = 10, 
    dy = 135, 
    dx = 20
)


rate_points = alt.Chart(all_uninsured).add_selection(
    nearest_val
).mark_point(
    filled = True, 
    size = 50
).encode(
    x = alt.X('year:O'), 
    y = alt.Y('percent_new:Q'), 
    opacity = opacity_condition, 
    color = alt.Color('disability_status:N')
)

rate_labels = alt.Chart(all_uninsured).mark_text(
    fontSize = 12, 
    dx = 7, 
    dy = 10, 
).encode(
    x = alt.X('year:O'), 
    y = alt.Y('percent_new:Q'), 
    text = alt.Text('percent_new:Q', format = '.1%')
).transform_filter(nearest_val)

c1 = alt.layer(uninsured_rates, aca_passage, aca_effect, aca_passage_text, aca_effect_text, rate_points, rate_labels)

c1

alt.LayerChart(...)

In [443]:
#visualization 2

insured_lines = alt.Chart(disabled).mark_line().encode(
    x = alt.X('year:O'),
    y = alt.Y('percent_new:Q'), 
    color = alt.Color('type:N')
)

insured_points = alt.Chart(disabled).add_selection(
    nearest_val
).mark_point(
    filled = True, 
    size = 50
).encode(
    x = alt.X('year:O'), 
    y = alt.Y('percent_new:Q'), 
    opacity = opacity_condition, 
    color = alt.Color('disability_status:N')
)

insured_labels = alt.Chart(disabled).mark_text(
    fontSize = 12, 
    dx = 7, 
    dy = 10, 
).encode(
    x = alt.X('year:O'), 
    y = alt.Y('percent_new:Q'), 
    text = alt.Text('percent_new:Q', format = '.1%')
).transform_filter(nearest_val)

c2 = alt.layer(insured_points, insured_lines, insured_labels)
c2

alt.LayerChart(...)

In [445]:
alt.Chart(disabled).mark_bar().encode(
    x = alt.X('percent_new:Q'), 
    y = alt.Y('year:O'), 
    color = alt.Color('type')
)

alt.Chart(...)

In [438]:
alt.hconcat(c1, c2)


alt.HConcatChart(...)